# Iteration 2: Post stroke unit destination

## Plain English Summary

In the model length of stay is determined not just by patient type, but also patient destination post ASU.  For example Early Supported Discharge greatly reduces length of stay.  As such the model is designed to sample destination as the patient arrives.  



## Notebook imports

In [1]:
from IPython.display import Markdown, display

## LLM Prompt

In [2]:
display(Markdown("inputs/02_iteration.md"))

### Main command

Modify the model to sample a patients destination after they have left the acute stroke unit (ASU).

Show all of the model code: this includes all patient generator functions in the ASU class.

### General model logic

After patients have arrived the model immediately samples their destination after the ASU. Destinations are:

1. Rehab
2. ESD 
3. Other

Create a variable called "post_asu_destination". This variable is used to record the sampled destination of the patient.

### Simulation inputs

The sampling distribution are specific to the patient type.

Stroke = Discrete Empirical: 24, 13, 63
TIA = Discrete Empirical: 1, 1, 98
Complex-neurological = Discrete Empirical: 11, 5, 84
Other = Discrete Empirical 5, 10, 85




## Date and time of input to LLM

> 1645 9th February 2024

## Observations on the LLM Output

* The following line of code has been inserted into each generator function. The choice function works as a discrete-empirical. The percentages were converted into probabilities. The proportions for each patient type are correct.

```python
post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.24, 0.13, 0.63])
```

## Sources reported by the LLM

* http://essay.utwente.nl/90598/1/Nijs_BA_BMS.pdf
* https://journals.sagepub.com/doi/pdf/10.1177/23814683221134098
* https://journals.sagepub.com/doi/pdf/10.1177/1460458216628314
* https://scholarworks.iupui.edu/server/api/core/bitstreams/21e08d17-9227-4b38-90e7-7fc8395582ea/content
* https://www.researchgate.net/publication/360946800_Improving_emergency_treatment_for_patients_with_acute_stroke_the_PEARS_research_programme_including_the_PASTA_cluster_RCT

## LLM Output

In [9]:
import simpy
import numpy as np

# Define the discrete-event simulation model
class AcuteStrokeUnit:
    def __init__(self, env, run_length=1825):  # 5 years in days
        self.env = env
        self.stroke_arrival = simpy.Container(env, init=0)
        self.tia_arrival = simpy.Container(env, init=0)
        self.neuro_arrival = simpy.Container(env, init=0)
        self.other_arrival = simpy.Container(env, init=0)
        self.stroke_interarrival_mean = 1.2
        self.tia_interarrival_mean = 9.3
        self.neuro_interarrival_mean = 3.6
        self.other_interarrival_mean = 3.2
        self.run_length = run_length
        self.patient_count = 0

    def stroke_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.stroke_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.stroke_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.24, 0.13, 0.63])
            print(f"Stroke patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")

    def tia_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.tia_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.tia_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.01, 0.01, 0.98])
            print(f"TIA patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")

    def neuro_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.neuro_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.neuro_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.11, 0.05, 0.84])
            print(f"Complex Neuro patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")

    def other_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.other_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.other_arrival.put(1)
            self.patient_count += 1
            post_asu_destination = np.random.choice(["Rehab", "ESD", "Other"], p=[0.05, 0.10, 0.85])
            print(f"Other patient {self.patient_count} arrived at {self.env.now} days and will go to {post_asu_destination}")

# Create the simulation environment
env = simpy.Environment()

# Initialize the Acute Stroke Unit model
acu = AcuteStrokeUnit(env)

# Start the patient generators
env.process(acu.stroke_patient_generator())
env.process(acu.tia_patient_generator())
env.process(acu.neuro_patient_generator())
env.process(acu.other_patient_generator())

# Run the simulation
env.run(until=acu.run_length)


Complex Neuro patient 1 arrived at 0.3180559558776053 days and will go to Other
Other patient 2 arrived at 0.788342961447337 days and will go to Other
Other patient 3 arrived at 1.1035420582449855 days and will go to Other
Stroke patient 4 arrived at 1.567257414416791 days and will go to Other
Complex Neuro patient 5 arrived at 2.65847389984878 days and will go to Rehab
Other patient 6 arrived at 2.7390288249374404 days and will go to Other
Stroke patient 7 arrived at 2.9128485871531735 days and will go to Other
TIA patient 8 arrived at 3.0641072516795904 days and will go to Other
Other patient 9 arrived at 3.3293995748948646 days and will go to Other
Complex Neuro patient 10 arrived at 4.03097446307214 days and will go to Other
Stroke patient 11 arrived at 4.0804170804417215 days and will go to Other
Stroke patient 12 arrived at 4.188724513334744 days and will go to Other
Stroke patient 13 arrived at 4.842462163714386 days and will go to Other
Complex Neuro patient 14 arrived at 5.439

## Testing

* **Extreme value test 1**: TIA, Compelex Neuro, Other, have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods

### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [10]:
M = 10_000_000

In [11]:
def test_1(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = 1.2
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [8]:
test_1(M)

Stroke patient 1 arrived at 1.3950230220880395 days and will go to Other
Stroke patient 2 arrived at 2.5573961110642793 days and will go to ESD
Stroke patient 3 arrived at 3.87049689352769 days and will go to Rehab
Stroke patient 4 arrived at 4.140490052742287 days and will go to Rehab
Stroke patient 5 arrived at 4.459390395604585 days and will go to ESD
Stroke patient 6 arrived at 5.078693924373392 days and will go to Rehab
Stroke patient 7 arrived at 5.497595963487679 days and will go to Other
Stroke patient 8 arrived at 6.588610622748083 days and will go to Rehab
Stroke patient 9 arrived at 6.611728095871669 days and will go to Other
Stroke patient 10 arrived at 7.582918598929445 days and will go to Other
Stroke patient 11 arrived at 7.8082390554854255 days and will go to Other
Stroke patient 12 arrived at 10.432680663283575 days and will go to Other
Stroke patient 13 arrived at 13.0199207844317 days and will go to Other
Stroke patient 14 arrived at 13.372563782775455 days and will 

### Extreme value test 2

* The code is again modified so that all patient type also have a mean inter-arrival rate of $M$

**Result: PASS**

In [12]:
def test_2(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = large_number
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [13]:
test_2(M)

0


### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [14]:
TEN_DAYS = 10

In [15]:
def test_3(run_length):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env, run_length=run_length)
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(env.now)
    


In [16]:
test_3(TEN_DAYS)

Other patient 1 arrived at 0.11769030790084523 days and will go to Other
Stroke patient 2 arrived at 1.457373494786602 days and will go to ESD
Complex Neuro patient 3 arrived at 1.8927591050240469 days and will go to Rehab
Stroke patient 4 arrived at 2.342862527512407 days and will go to Other
Stroke patient 5 arrived at 3.1126955013875444 days and will go to Other
Other patient 6 arrived at 3.2344259656687497 days and will go to Other
Other patient 7 arrived at 3.5082346060581173 days and will go to Other
Complex Neuro patient 8 arrived at 3.831017503976723 days and will go to Other
Stroke patient 9 arrived at 5.139056166479515 days and will go to Rehab
Stroke patient 10 arrived at 5.434637445008187 days and will go to Other
Stroke patient 11 arrived at 5.582532681139645 days and will go to Other
Other patient 12 arrived at 5.982397320615894 days and will go to Other
TIA patient 13 arrived at 6.037429527899504 days and will go to Other
TIA patient 14 arrived at 6.328046612599138 days 